<a href="https://colab.research.google.com/github/LincyNivi/RFM-Analysis-Using-Machine-Learning/blob/main/RFM_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is RFM Analysis?
* RFM (Recency, Frequency, Monetary) analysis is a powerful technique used by businesses to segment and understand their customers based on their buying behavior.
* It helps businesses identify their most valuable customers, understand their needs, and tailor marketing strategies to improve customer retention and loyalty.

* In this tutorial, I'll walk you through the steps of performing RFM analysis with some practical examples.

# Step 1: Data Preparation

> To conduct RFM analysis, you'll need transactional data that includes the following information:

1. Customer ID: A unique identifier for each customer.
1. Transaction Date: The date when the customer made a purchase.
1. Transaction Amount: The amount spent by the customer in that transaction.

# Step 2: Calculate RFM Metrics

> Now, we'll calculate the RFM metrics for each customer:

* Recency (R): The number of days since the customer's last purchase.
* Frequency (F): The total number of transactions made by the customer.
* Monetary Value (M): The total amount spent by the customer.

In [ ]:
import pandas as pd
from datetime import datetime

# Load the data
df = pd.read_excel("/content/RFM.xlsx")  ## reading the data
df.head()  ## top 5 rows of data

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [ ]:
df[df.CustomerID == 17850.0]

AttributeError: 'DataFrame' object has no attribute 'CustomerID'

In [ ]:
df.shape ## rows, column

(525461, 8)

In [ ]:
df.CustomerID.nunique()  ## nunique : gives the no of unique values in a column
#df.CustomerID.unique()   ## unique : gives the unique values in a column
#df.CustomerID.value_counts() ## value_counts : tells me how many times a value of a column is present in the table

4372

In [ ]:
df.info()

## 541910 - 406830 ## null values in customer ID column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   TransactionDate    541910 non-null  datetime64[ns]
 1   TransactionAmount  541910 non-null  float64       
 2   CustomerID         541910 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 12.4 MB


In [ ]:
df.fillna(-1,inplace=True)

In [ ]:
df.describe()  ## basic statistical analysis of my numeric variables

,TransactionAmount,CustomerID
count,541910.000000,541910.000000
mean,4.611138,11476.727625
std,96.759765,6778.324352
min,-11062.060000,-1.000000
25%,1.250000,12352.000000
50%,2.080000,14382.000000
75%,4.130000,16255.000000
max,38970.000000,18287.000000


In [ ]:
# Convert TransactionDate to a datetime object

df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

In [ ]:
#5 mar 2023 , 7 jul 23, 8 oct 23 , 5 jan 2024 , 3 mar 2024  ---> max txn date : 3 mar 2024 :

## 24 may 2024 - 3 mar 2024 : convert it to days ---> recency value


# Calculate Recency, Frequency, and Monetary Value for each customer
today = datetime(2024, 6, 21)   ## today's date to calculate the recency : recency = last order date - today's date

rfm = df.groupby('CustomerID').agg({
    'TransactionDate': lambda x: (today - x.max()).days,   ## give me the diff of max(txn date) of a customer & today's date
    'CustomerID': 'count',      ### Frequency of the customer txn
    'TransactionAmount': 'sum'  ### total spend of the customer
})

# Rename the columns for clarity
rfm.rename(columns={
    'TransactionDate': 'Recency',
    'CustomerID': 'Frequency',
    'TransactionAmount': 'Monetary'
}, inplace=True)

print(rfm.head())

            Recency  Frequency  Monetary
CustomerID                              
12346.0        4872          2      2.08
12347.0        4549        182    481.21
12348.0        4622         31    178.71
12349.0        4565         73    605.10
12350.0        4857         17     65.30


In [ ]:
df[df.CustomerID == 12346]  ## this particular customer

,TransactionDate,TransactionAmount,CustomerID
61619,2011-01-18 10:01:00,1.04,12346.0
61624,2011-01-18 10:17:00,1.04,12346.0


In [ ]:
x = 4872/366
x

13.311475409836065

In [ ]:
rfm.shape  ## 4372 customers

(4372, 3)

# Step 3: RFM Segmentation

> Next, we'll segment the customers based on their RFM scores.
* To do this, we'll divide each RFM metric into quartiles (or any other relevant segments) to create meaningful segments.

In [ ]:
# Create quartiles for Recency, Frequency, and Monetary
quantiles = rfm.quantile(q=[0.25, 0.5, 0.75])

# Create a function to assign RFM scores and segments
def rfm_segment(row):
    r_score = 1 if row['Recency'] <= quantiles['Recency'][0.25] else 2 if row['Recency'] <= quantiles['Recency'][0.5] else 3 if row['Recency'] <= quantiles['Recency'][0.75] else 4
    f_score = 4 if row['Frequency'] <= quantiles['Frequency'][0.25] else 3 if row['Frequency'] <= quantiles['Frequency'][0.5] else 2 if row['Frequency'] <= quantiles['Frequency'][0.75] else 1
    m_score = 4 if row['Monetary'] <= quantiles['Monetary'][0.25] else 3 if row['Monetary'] <= quantiles['Monetary'][0.5] else 2 if row['Monetary'] <= quantiles['Monetary'][0.75] else 1
    return str(r_score) + str(f_score) + str(m_score)

## recency score is better if it is lower :
## frequency score is better if it is lower :
## monetary score is better if it is lower :

#recency value : less , r score is best at 1
#freqyency value : less , f score is best at 1
#monetary value : less , m score is best at 1

## best customer : 111 , worst customer : 444

# Assign RFM scores and segments to each customer
rfm['RFM_Segment'] = rfm.apply(rfm_segment, axis=1)
print(rfm.head())

            Recency  Frequency  Monetary RFM_Segment
CustomerID                                          
12346.0        4872          2      2.08         444
12347.0        4549        182    481.21         111
12348.0        4622         31    178.71         332
12349.0        4565         73    605.10         221
12350.0        4857         17     65.30         443


In [ ]:
##   R     F     M
A : 1/2 , 3/4 , 3/4
B : 1/2 , 3/4 , 1/2
C : 3/4 , 3/4 , 3/4
D : 3/4 , 1/2 , 1/2

# Step 4: Interpretation of RFM Segments

> Now that we have segmented the customers, we can interpret each segment based on their RFM scores:

1. High-Value Customers: High Recency (1 or 2), High Frequency (3 or 4), and High Monetary Value (3 or 4).
1. Potential Loyalists: High Recency (1 or 2), High Frequency (3 or 4), and Low Monetary Value (1 or 2).
1. At Risk: Low Recency (3 or 4), High Frequency (3 or 4), and High Monetary Value (3 or 4).
1. Lost Customers: Low Recency (3 or 4), Low Frequency (1 or 2), and Low Monetary Value (1 or 2).

# Step 5: Actionable Insights

> Once we understand our RFM segments, we can tailor marketing strategies accordingly:

1. High-Value Customers: Reward them with exclusive offers or loyalty programs to maintain their loyalty.
1. Potential Loyalists: Encourage them to increase their spending with personalized offers or discounts.
1. At Risk: Engage them with retention campaigns to prevent churn.
1. Lost Customers: Launch win-back campaigns to re-engage with these customers.
Remember that RFM analysis is a valuable tool for understanding customer behavior, and the segments and actions can be further refined based on your specific business goals and data.